# Plan for Medal-Winning Score

My baseline submission did not achieve a medal. Based on expert feedback, I am overhauling my approach to build a robust, high-performing pipeline. This notebook will house the development of this new strategy.

## Core Strategy: 5-Fold Cross-Validation with Improved Models

1.  **New DICOM Preprocessing (High Priority):**
    *   **Action:** Discard the single-channel PNGs. Create new 3-channel PNGs by stacking different windowing settings (e.g., lung, mediastinum, soft tissue).
    *   **Rationale:** This provides the models with much richer information from the raw DICOMs, a key technique in many high-scoring solutions.
    *   **Implementation:** Create a new directory `train_png_3ch/` and `test_png_3ch/` and write a new preprocessing function to generate these images.

2.  **Full 5-Fold Cross-Validation:**
    *   **Action:** Train both the detector and classifier on all 5 folds created with `StratifiedGroupKFold`.
    *   **Rationale:** This makes the models more robust and allows for ensembling, which consistently improves scores.

3.  **Upgraded Classifier:**
    *   **Model:** Upgrade from `EfficientNet-B4` to a larger `EfficientNet-B5` or `B6`.
    *   **Aggregation:** Move from simple image-level averaging to a more sophisticated method. I will start by implementing the expert's suggestion: `Typical`/`Atypical` = max, `Indeterminate` = mean, `Negative` = min, then re-normalize.
    *   **Training:** Train for more epochs with augmentations (Mixup/Cutmix).

4.  **Upgraded Detector:**
    *   **Model:** Replace the baseline YOLOv5s with a stronger model. The expert suggested a segmentation-to-box approach (UNet) or a better detector (YOLOX, Detectron2). I will start by upgrading to **YOLOX**, as it's a powerful successor to YOLOv5.
    *   **Training:** Train on a larger image size (e.g., 1024x1024) for more epochs.

5.  **Robust Validation:**
    *   **Action:** Implement an offline evaluation function that calculates the official competition **blended mAP** on out-of-fold (OOF) predictions.
    *   **Rationale:** This is critical for reliably tuning hyperparameters (e.g., confidence thresholds, NMS IoU) and evaluating model performance without burning through submission attempts.

6.  **Ensembling and Post-Processing:**
    *   **Detection:** Use **Weighted Boxes Fusion (WBF)** to combine predictions from all 5 detector folds.
    *   **Classification:** Average the logits from all 5 classifier folds.
    *   **Fusion:** Implement classifier-guided filtering: suppress low-confidence bounding boxes for studies with a high probability of being `negative`.

## Execution Plan

1.  **Setup:** Basic imports, define constants, and copy the data loading/fold creation logic.
2.  **3-Channel Preprocessing:** Write and execute the new DICOM conversion script for all train and test images.
3.  **Classifier CV:** Implement and run the 5-fold training loop for the upgraded classifier.
4.  **Detector CV:** Set up YOLOX and run its 5-fold training loop.
5.  **Inference Pipeline:** Build a new inference script that loads all 10 models (5 detector, 5 classifier), performs TTA, WBF, and generates the final submission.

In [1]:
# --- Setup, Data Loading, and Fold Creation ---
import os
import glob
import pandas as pd
import numpy as np
import cv2
import pydicom
import SimpleITK as sitk
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.auto import tqdm

# --- Configuration ---
DATA_DIR = './'
TRAIN_DIR = 'train/'
TEST_DIR = 'test/'
TRAIN_PNG_3CH_DIR = 'train_png_3ch/'
TEST_PNG_3CH_DIR = 'test_png_3ch/'

N_SPLITS = 5
RANDOM_STATE = 42

os.makedirs(TRAIN_PNG_3CH_DIR, exist_ok=True)
os.makedirs(TEST_PNG_3CH_DIR, exist_ok=True)

print("Configuration and directories set up.")

# --- Load Data and Create Folds ---
print("Loading and preparing metadata...")

# Load original data
df_study = pd.read_csv(os.path.join(DATA_DIR, 'train_study_level.csv'))
df_image = pd.read_csv(os.path.join(DATA_DIR, 'train_image_level.csv'))

# Clean up IDs and merge
df_study['StudyInstanceUID'] = df_study['id'].apply(lambda x: x.replace('_study', ''))
df_image['image_id'] = df_image['id'].apply(lambda x: x.replace('_image', ''))
df_merged = df_image.merge(df_study, on='StudyInstanceUID', how='left')

# Find all train dcm files
train_dcm_files = glob.glob(f'{TRAIN_DIR}/*/*/*.dcm')
image_id_to_path = {os.path.basename(p).replace('.dcm', ''): p for p in train_dcm_files}
df_merged['dcm_path'] = df_merged['image_id'].map(image_id_to_path)

# Create a single target column for stratification
label_cols = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
df_merged['label_name'] = df_merged[label_cols].idxmax(axis=1)

# Create Folds
df_folds = df_merged.drop_duplicates('StudyInstanceUID').reset_index(drop=True)
sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
groups = df_folds['StudyInstanceUID']
y_stratify = df_folds['label_name']
df_folds['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(sgkf.split(df_folds, y_stratify, groups)):
    df_folds.loc[val_idx, 'fold'] = fold

# Merge fold info back into the main dataframe
df_merged = df_merged.merge(df_folds[['StudyInstanceUID', 'fold']], on='StudyInstanceUID', how='left')

print("Data loaded and folds created.")
print(f"Total images: {len(df_merged)}")
print("Fold distribution (by study):")
print(df_folds['fold'].value_counts())
display(df_merged.head())

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration and directories set up.
Loading and preparing metadata...


Data loaded and folds created.
Total images: 5696
Fold distribution (by study):
fold
1    1091
0    1090
3    1089
4    1089
2    1089
Name: count, dtype: int64


,id_x,boxes,label,StudyInstanceUID,image_id,id_y,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,dcm_path,label_name,fold
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2,5776db0cec75_study,0,1,0,0,train/5776db0cec75/81456c9c5423/000a312787f2.dcm,Typical Appearance,1
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,ff0879eb20ed_study,1,0,0,0,train/ff0879eb20ed/d8a644cc4f93/000c3a3f293f.dcm,Negative for Pneumonia,3
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc,9d514ce429a7_study,0,1,0,0,train/9d514ce429a7/22897cd1daa0/0012ff7358bc.dcm,Typical Appearance,4
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,001398f4ff4f,28dddc8559b2_study,0,0,0,1,train/28dddc8559b2/4d47bc042ee6/001398f4ff4f.dcm,Atypical Appearance,3
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,001bd15d1891,dfd9fdd85a3e_study,0,1,0,0,train/dfd9fdd85a3e/49170afa4f27/001bd15d1891.dcm,Typical Appearance,0


In [ ]:
# --- 3-Channel DICOM Preprocessing Function ---

def window_image(img, window_center, window_width, intercept, slope):
    """Applies windowing to a raw DICOM image."""
    img = (img * slope + intercept) # Apply rescale
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    w_img = np.clip(img, img_min, img_max)
    w_img = (w_img - img_min) / (img_max - img_min + 1e-6) # Normalize to [0, 1]
    w_img = (w_img * 255).astype(np.uint8) # Scale to [0, 255]
    return w_img

def process_dicom_to_3ch(dcm_path, output_dir):
    """
    Reads a DICOM file, applies three different windowing settings,
    stacks them into a 3-channel image, and saves as PNG.
    """
    image_id = os.path.basename(dcm_path).replace('.dcm', '')
    save_path = os.path.join(output_dir, f"{image_id}.png")
    if os.path.exists(save_path):
        return

    try:
        dicom = pydicom.dcmread(dcm_path)
        
        # Get rescale slope and intercept
        slope = float(dicom.RescaleSlope) if 'RescaleSlope' in dicom else 1.0
        intercept = float(dicom.RescaleIntercept) if 'RescaleIntercept' in dicom else 0.0
        
        # Get raw pixel data
        data = dicom.pixel_array
        
        # Handle MONOCHROME1
        if dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
            
        # Apply windowing based on expert advice
        # Window 1: Lung (center -600, width 1500)
        window1 = window_image(data, -600, 1500, intercept, slope)
        # Window 2: Mediastinum (center 50, width 400)
        window2 = window_image(data, 50, 400, intercept, slope)
        # Window 3: Soft tissue (center 40, width 350)
        window3 = window_image(data, 40, 350, intercept, slope)
        
        # Stack the windows into a 3-channel image
        img_3ch = np.stack([window1, window2, window3], axis=-1)
        
        # Save the image
        cv2.imwrite(save_path, img_3ch)

    except Exception as e:
        print(f"Error processing {dcm_path}: {e}")

print("3-channel DICOM processing function defined.")

In [ ]:
# --- Run 3-Channel Preprocessing on Training Data ---
print(f"Starting 3-channel preprocessing for {df_merged['dcm_path'].nunique()} training images...")
print(f"Output directory: {TRAIN_PNG_3CH_DIR}")

# Use tqdm to show a progress bar
for dcm_path in tqdm(df_merged['dcm_path'].unique(), desc="Processing Train DICOMs"):
    if pd.notna(dcm_path):
        process_dicom_to_3ch(dcm_path, TRAIN_PNG_3CH_DIR)

print("Finished preprocessing training images.")

In [ ]:
# --- Run 3-Channel Preprocessing on Test Data ---
print("Starting 3-channel preprocessing for test images...")
print(f"Output directory: {TEST_PNG_3CH_DIR}")

# Find all test dcm files
test_dcm_files = glob.glob(f'{TEST_DIR}/*/*/*.dcm')

for dcm_path in tqdm(test_dcm_files, desc="Processing Test DICOMs"):
    if pd.notna(dcm_path):
        process_dicom_to_3ch(dcm_path, TEST_PNG_3CH_DIR)

print("Finished preprocessing test images.")

In [ ]:
# --- Classifier: Dataset, Augmentations, and Model ---
import torch
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

# --- Configuration for Classifier ---
IMG_SIZE_CLS = 512
BATCH_SIZE_CLS = 16
MODEL_NAME_CLS = 'tf_efficientnet_b5_ns' # Upgrading from B4
NUM_CLASSES = 4
NUM_WORKERS = 4

# --- Dataset Class for 3-Channel Images ---
class SIIMCVClassifierDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.image_ids = df['image_id'].values
        self.labels = df['label_id'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = os.path.join(self.img_dir, f"{image_id}.png")
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label

# --- Augmentations ---
def get_cls_transforms(img_size, is_train=True):
    if is_train:
        # Using more robust augmentations as per expert advice
        return A.Compose([
            A.Resize(img_size, img_size),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=20, p=0.75),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.75),
            A.CoarseDropout(max_holes=8, max_height=img_size//8, max_width=img_size//8, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(img_size, img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# --- Add label_id to dataframe for dataset ---
label_map = {name: i for i, name in enumerate(label_cols)}
df_merged['label_id'] = df_merged['label_name'].map(label_map)

print("Classifier Dataset, Augmentations, and Config defined.")

In [ ]:
# --- Classifier: Training and Validation Loop ---
import time
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import roc_auc_score

def train_one_epoch(model, train_loader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc="Training", leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    return total_loss / len(train_loader)

def validate_one_epoch(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Validating", leave=False)
        for images, labels in progress_bar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_preds.append(torch.softmax(outputs, dim=1).cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    avg_loss = total_loss / len(val_loader)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    try:
        # FIX: Use all possible labels to prevent error if a class is missing in a fold
        auc_score = roc_auc_score(all_labels, all_preds, multi_class='ovo', labels=range(NUM_CLASSES))
    except ValueError:
        auc_score = -1.0 # Handle cases with only one class in a batch
    return avg_loss, auc_score, all_preds, all_labels

print("Classifier training and validation functions defined.")

In [ ]:
# --- Main Classifier CV Training Loop ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS_CLS = 15 # Train for more epochs than baseline
LEARNING_RATE_CLS = 1e-4

oof_preds = []
oof_labels = []
oof_image_ids = []
oof_folds = []

for fold in range(N_SPLITS):
    print("-" * 50)
    print(f"FOLD {fold}")
    print("-" * 50)

    # --- DataLoaders ---
    df_train = df_merged[df_merged['fold'] != fold].reset_index(drop=True)
    df_val = df_merged[df_merged['fold'] == fold].reset_index(drop=True)

    train_dataset = SIIMCVClassifierDataset(df_train, TRAIN_PNG_3CH_DIR, transform=get_cls_transforms(IMG_SIZE_CLS, is_train=True))
    val_dataset = SIIMCVClassifierDataset(df_val, TRAIN_PNG_3CH_DIR, transform=get_cls_transforms(IMG_SIZE_CLS, is_train=False))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_CLS, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE_CLS, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    # --- Model, Optimizer, Loss ---
    model = timm.create_model(MODEL_NAME_CLS, pretrained=True, num_classes=NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE_CLS)
    criterion = torch.nn.CrossEntropyLoss()
    scaler = GradScaler()

    best_val_loss = float('inf')
    best_epoch_preds = None
    best_epoch_labels = None
    best_model_path = f'classifier_fold{fold}_best.pth'

    for epoch in range(NUM_EPOCHS_CLS):
        start_time = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, DEVICE, scaler)
        val_loss, val_auc, val_preds, val_labels = validate_one_epoch(model, val_loader, criterion, DEVICE)
        elapsed_time = time.time() - start_time

        print(f"Epoch {epoch+1}/{NUM_EPOCHS_CLS} | Time: {elapsed_time:.0f}s | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch_preds = val_preds
            best_epoch_labels = val_labels
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best model saved to {best_model_path} (Val Loss: {best_val_loss:.4f})")

    # Store OOF predictions from the best epoch of this fold
    oof_preds.append(best_epoch_preds)
    oof_labels.append(best_epoch_labels)
    oof_image_ids.extend(df_val['image_id'].tolist())
    oof_folds.extend(df_val['fold'].tolist())

print("\n--- Classifier CV Training Finished ---")

# Concatenate all OOF predictions and save them
oof_preds_all = np.concatenate(oof_preds)
oof_labels_all = np.concatenate(oof_labels)
df_oof = pd.DataFrame(oof_preds_all, columns=label_cols)
df_oof['image_id'] = oof_image_ids
df_oof['label'] = oof_labels_all
df_oof['fold'] = oof_folds
df_oof.to_csv('oof_classifier.csv', index=False)
print("OOF predictions saved to oof_classifier.csv")

In [ ]:
# --- Generate OOF Predictions from Saved Models ---
# The previous cell was interrupted after training completed. 
# This cell will load the saved models to generate the OOF file without retraining.

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

oof_preds = []
oof_labels = []
oof_image_ids = []
oof_folds = []

print("Generating OOF predictions from saved models...")

for fold in range(N_SPLITS):
    print(f"--- Processing Fold {fold} ---")
    
    # --- DataLoaders ---
    df_val = df_merged[df_merged['fold'] == fold].reset_index(drop=True)
    val_dataset = SIIMCVClassifierDataset(df_val, TRAIN_PNG_3CH_DIR, transform=get_cls_transforms(IMG_SIZE_CLS, is_train=False))
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE_CLS, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    # --- Load Model ---
    model = timm.create_model(MODEL_NAME_CLS, pretrained=False, num_classes=NUM_CLASSES).to(DEVICE)
    model_path = f'classifier_fold{fold}_best.pth'
    model.load_state_dict(torch.load(model_path))
    criterion = torch.nn.CrossEntropyLoss() # Needed for validate function

    # --- Validate and Get Predictions ---
    val_loss, val_auc, val_preds, val_labels = validate_one_epoch(model, val_loader, criterion, DEVICE)
    print(f"Fold {fold} | Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f}")

    # Store OOF predictions
    oof_preds.append(val_preds)
    # FIX: val_labels is already a 1D array of class indices, no need for argmax
    oof_labels.append(val_labels)
    oof_image_ids.extend(df_val['image_id'].tolist())
    oof_folds.extend(df_val['fold'].tolist())

print("\n--- OOF Generation Finished ---")

# Concatenate all OOF predictions and save them
oof_preds_all = np.concatenate(oof_preds)
oof_labels_all = np.concatenate(oof_labels)
df_oof = pd.DataFrame(oof_preds_all, columns=label_cols)
df_oof['image_id'] = oof_image_ids
df_oof['label'] = oof_labels_all
df_oof['fold'] = oof_folds
df_oof.to_csv('oof_classifier.csv', index=False)
print("OOF predictions saved to oof_classifier.csv")
display(df_oof.head())

# --- YOLOX Detector: Setup and CV Training ---

With the classifier models trained, the next step is to train the object detector. I will use YOLOX, a strong anchor-free detector.

## Plan:

1.  **Install YOLOX:** Clone the official YOLOX repository and install its dependencies.
2.  **Prepare Data in YOLOX Format:**
    *   Create a directory structure like `yolox_data/`. 
    *   Inside, create `images/train` and `labels/train`.
    *   Copy all the 3-channel training images into `yolox_data/images/train/`.
    *   For each image with bounding boxes, create a corresponding `.txt` file in `yolox_data/labels/train/`. Each line in the file will be `class_id x_center y_center width height` (normalized). Since we only have one class ('opacity'), the `class_id` will always be 0.
3.  **Create Fold-Specific Files:**
    *   For each of the 5 folds, I will generate a `train.txt` and `val.txt` file. These files will contain the paths to the images for that fold's training and validation sets, respectively.
    *   Example: `yolox_data/fold_0_train.txt`, `yolox_data/fold_0_val.txt`.
4.  **Create YOLOX Configuration File:**
    *   Create a Python configuration file (e.g., `siim_yolox_config.py`) that inherits from a base YOLOX config (like `yolox_s.py`).
    *   In this file, I will specify the number of classes (1), the depth/width of the model, and paths to the fold-specific `train.txt` and `val.txt` files.
5.  **Run 5-Fold CV Training:**
    *   Write a script or loop that iterates through each fold.
    *   For each fold, it will call the YOLOX `tools/train.py` script, passing the appropriate config file and specifying an output directory for the weights.
    *   I will use a medium-sized model like `yolox-m` and train for a reasonable number of epochs (e.g., 20-30) with an appropriate image size (e.g., 640x640 or 768x768 to start).

In [2]:
# --- YOLOX Installation ---
# The 'pip install -e .' command is failing due to environment conflicts.
# New approach: Clone the repo and install dependencies directly from requirements.txt.
!rm -rf YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX
# Remove onnx-simplifier from requirements as it causes build issues
!sed -i '/onnx-simplifier/d' requirements.txt
# Install all other dependencies
!pip install -r requirements.txt
!pip install pycocotools --no-deps
%cd ..

Cloning into 'YOLOX'...


/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/YOLOX
🔧 Intercepting pip install command: pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 463.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 384.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 240.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 KB 276.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 470.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 314.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 KB 498.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 KB 521.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 188.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.3/477.3 KB 514.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 319.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 541.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery-storage 2.24.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.32.1 which is incompatible.



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.3/477.3 KB 19.7 MB/s eta 0:00:00

✅ Package installation completed and import cache refreshed.
/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912


In [3]:
# --- YOLOX Data Prep Part 1: Get and Cache Image Dimensions & Create Symlinks ---
import os
import json
import cv2
from tqdm.auto import tqdm

IMAGE_DIMS_CACHE_PATH = 'image_dimensions.json'
image_dimensions = {}

# --- Step 1: Get Image Dimensions (from cache or by reading files) ---
if os.path.exists(IMAGE_DIMS_CACHE_PATH):
    print(f"Loading image dimensions from cache: {IMAGE_DIMS_CACHE_PATH}")
    with open(IMAGE_DIMS_CACHE_PATH, 'r') as f:
        image_dimensions = json.load(f)
else:
    print("Image dimensions cache not found. This will be slow, but only runs once.")
    unique_image_ids = df_merged['image_id'].unique()
    for image_id in tqdm(unique_image_ids, desc="Reading image dimensions"):
        img_path = os.path.join(TRAIN_PNG_3CH_DIR, f"{image_id}.png")
        if os.path.exists(img_path):
            try:
                h, w, _ = cv2.imread(img_path).shape
                image_dimensions[image_id] = (w, h)
            except Exception as e:
                print(f"Warning: Could not read dimensions for {image_id}. Error: {e}")

    print(f"Saving image dimensions to cache: {IMAGE_DIMS_CACHE_PATH}")
    with open(IMAGE_DIMS_CACHE_PATH, 'w') as f:
        json.dump(image_dimensions, f)

print(f"Loaded/created dimensions for {len(image_dimensions)} images.")

# --- Step 2: Create Image Symlinks for YOLOX ---
# The COCO data loader needs to find the actual image files.
YOLOX_DATA_DIR = 'yolox_data'
YOLOX_IMG_DIR = os.path.join(YOLOX_DATA_DIR, 'images', 'train')
os.makedirs(YOLOX_IMG_DIR, exist_ok=True)
print(f"Ensured YOLOX image directory exists: {YOLOX_IMG_DIR}")

print("Creating image symlinks for YOLOX...")
for image_id in tqdm(df_merged['image_id'].unique(), desc="Creating image symlinks"):
    src_path = os.path.abspath(os.path.join(TRAIN_PNG_3CH_DIR, f"{image_id}.png"))
    dst_path = os.path.join(YOLOX_IMG_DIR, f"{image_id}.png")
    if not os.path.lexists(dst_path): # Use lexists to avoid errors with broken symlinks
        os.symlink(src_path, dst_path)
print("Finished creating image symlinks.")

Loading image dimensions from cache: image_dimensions.json
Loaded/created dimensions for 5696 images.
Ensured YOLOX image directory exists: yolox_data/images/train
Creating image symlinks for YOLOX...


Creating image symlinks:   0%|          | 0/5696 [00:00<?, ?it/s]

Creating image symlinks: 100%|██████████| 5696/5696 [00:00<00:00, 154961.70it/s]

Finished creating image symlinks.


In [4]:
# --- YOLOX Data Preparation: Convert to COCO JSON (REVISED) ---
import json
import ast
from tqdm.auto import tqdm

YOLOX_ANNO_DIR = os.path.join(YOLOX_DATA_DIR, 'annotations')
os.makedirs(YOLOX_ANNO_DIR, exist_ok=True)
print(f"Created YOLOX annotations directory: {YOLOX_ANNO_DIR}")

if 'image_dimensions' not in globals() or len(image_dimensions) == 0:
    print("FATAL ERROR: image_dimensions dictionary is not populated. Please re-run cell 11.")
else:
    print(f"Generating COCO JSON annotations for each fold... (using {len(image_dimensions)} image dimensions)")
    for fold in range(N_SPLITS):
        print(f"--- Processing Fold {fold} ---")

        # --- Process Train Split ---
        df_train_fold = df_merged[df_merged['fold'] != fold].copy()
        coco_data_train = {
            "info": {}, # FIX: Add empty info dict to prevent KeyError during evaluation
            "images": [],
            "annotations": [],
            "categories": [{"id": 0, "name": "opacity", "supercategory": "none"}]
        }
        image_id_map = {img_id: i for i, img_id in enumerate(df_train_fold['image_id'].unique())}

        for img_id, coco_id in tqdm(image_id_map.items(), desc=f"Fold {fold} Train Images"):
            if img_id in image_dimensions:
                w, h = image_dimensions[img_id]
                coco_data_train['images'].append({
                    "id": coco_id,
                    "file_name": f"{img_id}.png",
                    "width": w,
                    "height": h
                })

        annot_id = 0
        for row in tqdm(df_train_fold.itertuples(), total=len(df_train_fold), desc=f"Fold {fold} Train Annots"):
            if pd.notna(row.boxes):
                coco_id = image_id_map[row.image_id]
                boxes = ast.literal_eval(row.boxes)
                for box in boxes:
                    coco_data_train['annotations'].append({
                        "id": annot_id,
                        "image_id": coco_id,
                        "category_id": 0,
                        "bbox": [box['x'], box['y'], box['width'], box['height']],
                        "area": box['width'] * box['height'],
                        "iscrowd": 0
                    })
                    annot_id += 1

        train_json_path = os.path.join(YOLOX_ANNO_DIR, f'instances_fold{fold}_train.json')
        with open(train_json_path, 'w') as f: json.dump(coco_data_train, f)
        print(f"Saved train annotations to {train_json_path}")

        # --- Process Validation Split ---
        df_val_fold = df_merged[df_merged['fold'] == fold].copy()
        coco_data_val = {
            "info": {}, # FIX: Add empty info dict to prevent KeyError during evaluation
            "images": [],
            "annotations": [],
            "categories": [{"id": 0, "name": "opacity", "supercategory": "none"}]
        }
        image_id_map_val = {img_id: i for i, img_id in enumerate(df_val_fold['image_id'].unique())}

        for img_id, coco_id in tqdm(image_id_map_val.items(), desc=f"Fold {fold} Val Images"):
            if img_id in image_dimensions:
                w, h = image_dimensions[img_id]
                coco_data_val['images'].append({
                    "id": coco_id,
                    "file_name": f"{img_id}.png",
                    "width": w,
                    "height": h
                })

        annot_id_val = 0
        for row in tqdm(df_val_fold.itertuples(), total=len(df_val_fold), desc=f"Fold {fold} Val Annots"):
            if pd.notna(row.boxes):
                coco_id = image_id_map_val[row.image_id]
                boxes = ast.literal_eval(row.boxes)
                for box in boxes:
                    coco_data_val['annotations'].append({
                        "id": annot_id_val,
                        "image_id": coco_id,
                        "category_id": 0,
                        "bbox": [box['x'], box['y'], box['width'], box['height']],
                        "area": box['width'] * box['height'],
                        "iscrowd": 0
                    })
                    annot_id_val += 1

        val_json_path = os.path.join(YOLOX_ANNO_DIR, f'instances_fold{fold}_val.json')
        with open(val_json_path, 'w') as f: json.dump(coco_data_val, f)
        print(f"Saved val annotations to {val_json_path}")

    print("Finished generating all COCO JSON files.")

Created YOLOX annotations directory: yolox_data/annotations
Generating COCO JSON annotations for each fold... (using 5696 image dimensions)
--- Processing Fold 0 ---


Fold 0 Train Images:   0%|          | 0/4553 [00:00<?, ?it/s]

Fold 0 Train Images: 100%|██████████| 4553/4553 [00:00<00:00, 1549676.71it/s]

Fold 0 Train Annots:   0%|          | 0/4553 [00:00<?, ?it/s]

Fold 0 Train Annots: 100%|██████████| 4553/4553 [00:00<00:00, 61541.60it/s]

...

Fold 4 Val Images:   0%|          | 0/1151 [00:00<?, ?it/s]

Fold 4 Val Images: 100%|██████████| 1151/1151 [00:00<00:00, 1350389.90it/s]

Fold 4 Val Annots:   0%|          | 0/1151 [00:00<?, ?it/s]

Fold 4 Val Annots: 100%|██████████| 1151/1151 [00:00<00:00, 63057.82it/s]

Saved val annotations to yolox_data/annotations/instances_fold4_val.json
Finished generating all COCO JSON files.


In [ ]:
# --- Verify a generated JSON file ---
import json
from pycocotools.coco import COCO

# Check the first fold's training file
json_path = 'yolox_data/annotations/instances_fold0_train.json'
print(f"Verifying {json_path}...")

try:
    with open(json_path, 'r') as f:
        data = json.load(f)

    print(f"Number of images in JSON: {len(data['images'])}")
    print(f"Number of annotations in JSON: {len(data['annotations'])}")

    # Try to load with pycocotools
    coco_api = COCO(json_path)
    img_ids = coco_api.getImgIds()
    print(f"Number of image IDs found by pycocotools: {len(img_ids)}")

    if len(img_ids) == 0:
        print("!!! WARNING: pycocotools found 0 images. The JSON file might be structured incorrectly.")
    else:
        print("Verification successful. pycocotools can read the image IDs.")
except Exception as e:
    print(f"An error occurred during verification: {e}")

In [5]:
# --- FIX: Create symlinks for YOLOX data loader ---
# The YOLOX COCODataset loader expects separate 'train' and 'val' directories for images.
# My images are all in one directory. I'll create symlinks to trick the loader.
# Using absolute paths to avoid any ambiguity with relative paths.
import os

YOLOX_DATA_DIR = 'yolox_data'
TRAIN_LINK_PATH = os.path.join(YOLOX_DATA_DIR, 'train')
VAL_LINK_PATH = os.path.join(YOLOX_DATA_DIR, 'val')

# The target is the directory containing all the image symlinks
IMG_DIR_TARGET = os.path.join(YOLOX_DATA_DIR, 'images', 'train')
IMG_DIR_TARGET_ABS = os.path.abspath(IMG_DIR_TARGET)

# Clean up old links if they exist from a previous run
# Use os.lexists() to check for symlinks without following them.
if os.path.lexists(TRAIN_LINK_PATH):
    os.remove(TRAIN_LINK_PATH)
if os.path.lexists(VAL_LINK_PATH):
    os.remove(VAL_LINK_PATH)

# Create the new symlinks with an absolute path target
try:
    os.symlink(IMG_DIR_TARGET_ABS, TRAIN_LINK_PATH)
    os.symlink(IMG_DIR_TARGET_ABS, VAL_LINK_PATH)
    print(f"Created symlink: {TRAIN_LINK_PATH} -> {os.readlink(TRAIN_LINK_PATH)}")
    print(f"Created symlink: {VAL_LINK_PATH} -> {os.readlink(VAL_LINK_PATH)}")
except Exception as e:
    print(f"Error creating symlinks: {e}")

Created symlink: yolox_data/train -> /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/yolox_data/images/train
Created symlink: yolox_data/val -> /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/yolox_data/images/train


In [6]:
# --- Download Pretrained YOLOX Weights ---
import os

weights_dir = 'YOLOX/weights'
weights_path = os.path.join(weights_dir, 'yolox_m.pth')
weights_url = 'https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth'

os.makedirs(weights_dir, exist_ok=True)

if not os.path.exists(weights_path):
    print(f"Downloading pretrained weights to {weights_path}...")
    # Use curl instead of wget as wget is not found in the environment
    !curl -L -o {weights_path} {weights_url}
    print("Download complete.")
else:
    print("Pretrained weights already exist.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


  8  193M    8 15.8M    0     0  31.0M      0  0:00:06 --:--:--  0:00:06 31.0M

 54  193M   54  104M    0     0  69.3M      0  0:00:02  0:00:01  0:00:01 88.8M

 90  193M   90  175M    0     0  70.0M      0  0:00:02  0:00:02 --:--:-- 79.9M

Download complete.


100  193M  100  193M    0     0  70.6M      0  0:00:02  0:00:02 --:--:-- 79.7M


In [ ]:
# --- FIX: Install Ninja for YOLOX Evaluation ---
# The previous run failed because ninja, a build tool dependency for PyTorch's C++ extensions,
# was not found. YOLOX uses these extensions for its fast COCO evaluation API.
!pip install ninja

In [ ]:
# --- DEBUG: Ninja Installation and PATH ---
import os

print("--- Verifying Ninja Installation Location ---")
!which ninja
!ls -l /app/.pip-target/bin/ninja

print("\n--- Checking System PATH --- ")
path_var = os.environ.get('PATH', '')
print(path_var)

print("\n--- Checking if ninja's path is in PATH ---")
ninja_path = "/app/.pip-target/bin"
if ninja_path in path_var:
    print("SUCCESS: Ninja's directory is in the PATH.")
else:
    print("!!! ERROR: Ninja's directory is NOT in the PATH.")

In [ ]:
# --- DEBUG: Find the ninja executable ---
!echo 'Searching for ninja executable in /app ...'
!find /app -name ninja 2>/dev/null
!echo 'Searching for ninja executable in /usr ...'
!find /usr -name ninja 2>/dev/null

In [7]:
# --- PATCH YOLOX: Disable Fast C++ COCO Eval ---
# The training continues to fail because the subshell process for training cannot find the `ninja` executable,
# which is required to build the C++ extensions for fast COCO evaluation.
# Instead of continuing to debug the PATH, I will patch the YOLOX source code to use the standard, slower, pure-python evaluation API from pycocotools.
# This will bypass the C++ extension compilation entirely, thus removing the `ninja` dependency.

import os

evaluator_path = 'YOLOX/yolox/evaluators/coco_evaluator.py'
line_to_replace = 'from yolox.layers import COCOeval_opt as COCOeval'
replacement_line = 'from pycocotools.cocoeval import COCOeval  # Patched to disable C++ extension'

try:
    with open(evaluator_path, 'r') as f:
        content = f.read()
    
    if line_to_replace in content:
        new_content = content.replace(line_to_replace, replacement_line)
        with open(evaluator_path, 'w') as f:
            f.write(new_content)
        print(f'SUCCESS: Patched {evaluator_path} to disable fast COCO eval.')
    elif replacement_line in content:
        print(f'INFO: Patch already applied to {evaluator_path}.')
    else:
        print(f'!!! ERROR: Could not find the line to patch in {evaluator_path}.')
except Exception as e:
    print(f'An error occurred during patching: {e}')

SUCCESS: Patched YOLOX/yolox/evaluators/coco_evaluator.py to disable fast COCO eval.


In [9]:
# --- Modify YOLOX Config for Fewer Epochs ---
# I am reducing the number of epochs from 5 to 2 to ensure training completes in time.
import re

config_path = 'siim_yolox_config.py'
target_config_path = 'YOLOX/exps/siim_yolox_config.py'

try:
    with open(config_path, 'r') as f:
        content = f.read()
    
    # Change max_epoch to 2
    new_content = re.sub(r'self\.max_epoch = \d+', 'self.max_epoch = 2', content)
    
    with open(config_path, 'w') as f:
        f.write(new_content)
    print(f"SUCCESS: Modified {config_path} to use 2 epochs.")

    # Also update the copy in the YOLOX directory to be safe
    with open(target_config_path, 'w') as f:
        f.write(new_content)
    print(f"SUCCESS: Updated {target_config_path} as well.")

except Exception as e:
    print(f"An error occurred during config modification: {e}")

SUCCESS: Modified siim_yolox_config.py to use 2 epochs.
SUCCESS: Updated YOLOX/exps/siim_yolox_config.py as well.


In [10]:
# --- YOLOX 5-Fold CV Training (Final Attempt) ---
import os
import shutil
import re

# Set env var to mitigate CUDA memory fragmentation.
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
print(f"Set PYTORCH_CUDA_ALLOC_CONF to: {os.environ.get('PYTORCH_CUDA_ALLOC_CONF')}")

# Define project root to easily return to it
PROJECT_ROOT = os.getcwd()
print(f"Project root set to: {PROJECT_ROOT}")

# --- Setup for training ---
BASE_CONFIG_PATH = 'siim_yolox_config.py'
YOLOX_EXPS_DIR = 'YOLOX/exps'
TARGET_CONFIG_PATH = os.path.join(YOLOX_EXPS_DIR, 'siim_yolox_config.py')

PRETRAINED_WEIGHTS_PATH = 'weights/yolox_m.pth' # This path is relative to the YOLOX directory
print(f"Relative pretrained weights path to be used inside YOLOX dir: {PRETRAINED_WEIGHTS_PATH}")

# --- Clean previous run's outputs ---
print("Cleaning previous YOLOX output directory...")
!rm -rf yolox_outputs/*

# --- Training Loop ---
BATCH_SIZE_DET = 16
DEVICES = 1

for fold in range(N_SPLITS):
    print("\n" + "="*50)
    print(f"STARTING YOLOX TRAINING FOR FOLD {fold}")
    print("="*50)

    # --- DYNAMICALLY MODIFY CONFIG FILE FOR THIS FOLD ---
    # This is a robust way to ensure each fold has a unique output directory.
    exp_name = f"yolox_m_fold_{fold}"
    train_ann_file = f"instances_fold{fold}_train.json"
    val_ann_file = f"instances_fold{fold}_val.json"

    with open(BASE_CONFIG_PATH, 'r') as f: config_content = f.read()
    config_content = re.sub(r'self\.exp_name = .*', f'self.exp_name = "{exp_name}"', config_content)
    config_content = re.sub(r'self\.train_ann = .*', f'self.train_ann = "{train_ann_file}"', config_content)
    config_content = re.sub(r'self\.val_ann = .*', f'self.val_ann = "{val_ann_file}"', config_content)
    with open(TARGET_CONFIG_PATH, 'w') as f: f.write(config_content)
    print(f"Dynamically modified {TARGET_CONFIG_PATH} for fold {fold} with exp_name: {exp_name}")

    # --- RUN TRAINING ---
    %cd YOLOX

    command = (
        f"PYTHONPATH=/app/.pip-target:. python3 tools/train.py "
        f"--fp16 "
        f"-f exps/siim_yolox_config.py "
        f"-d {DEVICES} "
        f"-b {BATCH_SIZE_DET * DEVICES} "
        f"-c {PRETRAINED_WEIGHTS_PATH}"
    )
    
    !{command}
    
    %cd {PROJECT_ROOT}
    
    print(f"FINISHED YOLOX TRAINING FOR FOLD {fold}")

# Restore the original config file after the loop finishes
shutil.copy(BASE_CONFIG_PATH, TARGET_CONFIG_PATH)
print(f"\nRestored original config at {TARGET_CONFIG_PATH}")

print("\n" + "="*50)
print("ALL YOLOX FOLDS TRAINED")
print("="*50)

Set PYTORCH_CUDA_ALLOC_CONF to: expandable_segments:True
Project root set to: /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912
Relative pretrained weights path to be used inside YOLOX dir: weights/yolox_m.pth
Cleaning previous YOLOX output directory...

STARTING YOLOX TRAINING FOR FOLD 0
Dynamically modified YOLOX/exps/siim_yolox_config.py for fold 0 with exp_name: yolox_m_fold_0
/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/YOLOX


/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/YOLOX/yolox/core/trainer.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=args.fp16)
2025-09-26 18:17:53 | INFO     | yolox.core.trainer:132 - args: Namespace(experiment_name='yolox_m_fold_0', name=None, dist_backend='nccl', dist_url=None, batch_size=16, devices=1, exp_file='exps/siim_yolox_config.py', resume=False, ckpt='weights/yolox_m.pth', start_epoch=None, num_machines=1, machine_rank=0, fp16=True, cache=None, occupy=False, logger='tensorboard', opts=[])
2025-09-26 18:17:53 | INFO     | yolox.core.trainer:133 - exp value:
╒═══════════════════╤════════════════════════════════════════════════════════════════════════════════════════╕
│ keys              │ values                                                                                 │
╞═══════════════════╪══════════════════

2025-09-26 18:17:53 | INFO     | yolox.core.trainer:138 - Model Summary: Params: 25.28M, Gflops: 73.73
2025-09-26 18:17:53 | INFO     | yolox.core.trainer:338 - loading checkpoint for fine tuning


2025-09-26 18:17:53 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.0.weight in checkpoint is torch.Size([80, 192, 1, 1]), while shape of head.cls_preds.0.weight in model is torch.Size([1, 192, 1, 1]).
2025-09-26 18:17:53 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.0.bias in checkpoint is torch.Size([80]), while shape of head.cls_preds.0.bias in model is torch.Size([1]).
2025-09-26 18:17:53 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.1.weight in checkpoint is torch.Size([80, 192, 1, 1]), while shape of head.cls_preds.1.weight in model is torch.Size([1, 192, 1, 1]).
2025-09-26 18:17:53 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.1.bias in checkpoint is torch.Size([80]), while shape of head.cls_preds.1.bias in model is torch.Size([1]).
2025-09-26 18:17:53 | WARNING  | yolox.utils.checkpoint:24 - Shape of head.cls_preds.2.weight in checkpoint is torch.Size([80, 192, 1, 1]), while shape of head.cls_preds.2.weig

2025-09-26 18:17:55 | INFO     | yolox.data.datasets.coco:63 - loading annotations into memory...
2025-09-26 18:17:55 | INFO     | yolox.data.datasets.coco:63 - Done (t=0.00s)
2025-09-26 18:17:55 | INFO     | pycocotools.coco:86 - creating index...
2025-09-26 18:17:55 | INFO     | pycocotools.coco:86 - index created!
2025-09-26 18:17:55 | INFO     | yolox.core.trainer:196 - Training start...
2025-09-26 18:17:55 | INFO     | yolox.core.trainer:197 - 
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, mom

...

2025-09-26 19:11:34 | INFO     | yolox.evaluators.coco_evaluator:301 - Evaluate annotation type *bbox*


2025-09-26 19:11:35 | INFO     | yolox.evaluators.coco_evaluator:301 - DONE (t=0.78s).
2025-09-26 19:11:35 | INFO     | yolox.evaluators.coco_evaluator:302 - Accumulating evaluation results...
2025-09-26 19:11:35 | INFO     | yolox.evaluators.coco_evaluator:302 - DONE (t=0.11s).
2025-09-26 19:11:35 | INFO     | yolox.core.trainer:381 - 
Average forward time: 5.62 ms, Average NMS time: 0.48 ms, Average inference time: 6.09 ms
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.022
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.096
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.022
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.041
 Average 

2025-09-26 19:11:36 | INFO     | yolox.core.trainer:402 - Save weights to /var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912/yolox_outputs/yolox_m_fold_4


2025-09-26 19:11:36 | INFO     | yolox.core.trainer:200 - Training of experiment is done and the best AP is 2.24


/var/lib/simon/agent_run_states/siim-covid19-detection-20250925-205912
FINISHED YOLOX TRAINING FOR FOLD 4

Restored original config at YOLOX/exps/siim_yolox_config.py

ALL YOLOX FOLDS TRAINED


In [ ]:
# --- Check Shared Memory Size ---
!df -h /dev/shm

In [ ]:
# --- Definitive JSON Verification ---
import json

# Check the first fold's VALIDATION file, since that's what's used for evaluation
json_path = 'yolox_data/annotations/instances_fold0_val.json'
print(f"Verifying keys in {json_path}...")

try:
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"Top-level keys: {list(data.keys())}")

    if 'info' in data.keys():
        print("SUCCESS: 'info' key found in the JSON file.")
    else:
        print("!!! ERROR: 'info' key is MISSING from the JSON file.")

except Exception as e:
    print(f"An error occurred during verification: {e}")

In [11]:
# --- Verify YOLOX Checkpoint Creation ---
import os

print("Checking for saved YOLOX model checkpoints...")
for fold in range(N_SPLITS):
    exp_name = f"yolox_m_fold_{fold}"
    checkpoint_path = os.path.join('yolox_outputs', exp_name, 'best_ckpt.pth')
    if os.path.exists(checkpoint_path):
        print(f"SUCCESS: Found checkpoint for fold {fold} at: {checkpoint_path}")
    else:
        print(f"!!! ERROR: Checkpoint for fold {fold} is MISSING.")

Checking for saved YOLOX model checkpoints...
SUCCESS: Found checkpoint for fold 0 at: yolox_outputs/yolox_m_fold_0/best_ckpt.pth
SUCCESS: Found checkpoint for fold 1 at: yolox_outputs/yolox_m_fold_1/best_ckpt.pth
SUCCESS: Found checkpoint for fold 2 at: yolox_outputs/yolox_m_fold_2/best_ckpt.pth
SUCCESS: Found checkpoint for fold 3 at: yolox_outputs/yolox_m_fold_3/best_ckpt.pth
SUCCESS: Found checkpoint for fold 4 at: yolox_outputs/yolox_m_fold_4/best_ckpt.pth


# --- Offline Validation: Detector OOF & Blended mAP ---

To reliably estimate our final score, we need to calculate the blended mAP on our out-of-fold (OOF) predictions. We already have `oof_classifier.csv`. Now we need to generate OOF predictions for the detector.

## Plan:

1.  **Generate Detector OOF Predictions:**
    *   Loop through each fold (0 to 4).
    *   Load the best detector model for that fold (`yolox_outputs/yolox_m_fold_X/best_ckpt.pth`).
    *   Load the validation data for that fold (`instances_foldX_val.json`).
    *   Run inference on the validation set.
    *   Store the predictions (boxes, scores) in a dictionary or dataframe, keyed by `image_id`.
    *   Save the collected OOF predictions to a file (e.g., `oof_detector.pkl`).

2.  **Implement Blended mAP Calculation:**
    *   Write a function that takes the classifier OOF and detector OOF as input.
    *   **Study-level mAP:** Calculate the study-level score using the probabilities in `oof_classifier.csv` against the ground truth labels.
    *   **Image-level mAP:** Calculate the image-level mAP using the detector OOF predictions against the ground truth bounding boxes. This will require a standard mAP implementation (e.g., from `map_boxes` or a similar library).
    *   **Blend:** Average the two scores to get the final estimated CV score.

In [20]:
# --- Code for Detector OOF Generation ---
import torch
import pickle
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast
from tqdm.auto import tqdm

# Add YOLOX to path if not already there
import sys
if 'YOLOX' not in sys.path:
    sys.path.append('YOLOX')

from yolox.exp import get_exp
from yolox.utils import postprocess
from yolox.data import COCODataset, ValTransform

def generate_detector_oof(n_splits, config_path):
    """Generates and saves out-of-fold predictions for the detector models."""
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    IMG_SIZE_DET = 640
    BATCH_SIZE_DET = 16
    
    oof_predictions = {}

    for fold in range(n_splits):
        print(f"--- Generating OOF for Fold {fold} ---")
        
        # --- Load Model ---
        model_path = f'yolox_outputs/yolox_m_fold_{fold}/best_ckpt.pth'
        if not os.path.exists(model_path):
            print(f"!!! WARNING: Model for fold {fold} not found at {model_path}. Skipping.")
            continue
            
        exp = get_exp(config_path)
        exp.test_size = (IMG_SIZE_DET, IMG_SIZE_DET)
        model = exp.get_model().to(DEVICE)
        model.eval()
        ckpt = torch.load(model_path, map_location=DEVICE, weights_only=False)
        model.load_state_dict(ckpt["model"])
        
        # --- Validation Dataloader ---
        val_ann_file = f'instances_fold{fold}_val.json'
        val_dataset = COCODataset(
            data_dir=exp.data_dir,
            json_file=val_ann_file,
            name="val",
            img_size=exp.test_size,
            preproc=ValTransform(legacy=False),
        )
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE_DET, shuffle=False, num_workers=4, pin_memory=True)

        # --- Run Inference ---
        with torch.no_grad():
            for images, _, info_imgs, img_ids_coco_tensor in tqdm(val_loader, desc=f'Fold {fold} OOF Inference'):
                images = images.to(DEVICE, non_blocking=True)
                
                orig_heights = info_imgs[0].numpy()
                orig_widths = info_imgs[1].numpy()
                
                with autocast():
                    outputs = model(images)
                    outputs = postprocess(outputs, exp.num_classes, exp.test_conf, exp.nmsthre, class_agnostic=True)
                
                for i, output in enumerate(outputs):
                    coco_id = img_ids_coco_tensor[i].item()
                    original_image_id = val_dataset.coco.loadImgs(coco_id)[0]['file_name'].replace('.png', '')
                    
                    h, w = orig_heights[i], orig_widths[i]
                    ratio = min(IMG_SIZE_DET / h, IMG_SIZE_DET / w)
                    
                    if output is None:
                        oof_predictions[original_image_id] = {'boxes': [], 'scores': []}
                        continue
                    
                    output = output.cpu().numpy()
                    bboxes = output[:, 0:4] / ratio
                    # EXPERT FIX: Correct score calculation. YOLOX output is (box, obj_conf, class_conf, class_id)
                    # The score is obj_conf * class_conf.
                    scores = output[:, 4] * output[:, 5]
                    
                    oof_predictions[original_image_id] = {'boxes': bboxes.tolist(), 'scores': scores.flatten().tolist()}
    
    with open('oof_detector.pkl', 'wb') as f:
        pickle.dump(oof_predictions, f)
    print("Detector OOF predictions saved to 'oof_detector.pkl'")
    return oof_predictions

print("Detector OOF generation function defined.")

Detector OOF generation function defined.


In [21]:
# --- Run Detector OOF Generation ---
print("Starting detector OOF generation...")
oof_detector_preds = generate_detector_oof(
    n_splits=N_SPLITS,
    config_path='YOLOX/exps/siim_yolox_config.py'
)
print(f"Generated OOF predictions for {len(oof_detector_preds)} images.")

Starting detector OOF generation...
--- Generating OOF for Fold 0 ---


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Fold 0 OOF Inference:   0%|          | 0/72 [00:00<?, ?it/s]

/tmp/ipykernel_63363/1407663238.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Fold 0 OOF Inference:   1%|▏         | 1/72 [00:01<01:49,  1.54s/it]

Fold 0 OOF Inference:   3%|▎         | 2/72 [00:01<00:54,  1.28it/s]

...

Fold 4 OOF Inference:  94%|█████████▍| 68/72 [00:25<00:01,  3.13it/s]

Fold 4 OOF Inference:  97%|█████████▋| 70/72 [00:26<00:00,  2.05it/s]

Fold 4 OOF Inference: 100%|██████████| 72/72 [00:26<00:00,  2.91it/s]

Fold 4 OOF Inference: 100%|██████████| 72/72 [00:26<00:00,  2.69it/s]

Detector OOF predictions saved to 'oof_detector.pkl'
Generated OOF predictions for 5696 images.


In [24]:
# --- Code for Blended mAP Calculation ---

# Install a library for mAP calculation
!pip install -q map_boxes

import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import average_precision_score
from map_boxes import mean_average_precision_for_boxes

def calculate_blended_map(df_merged, oof_cls_path, oof_det_path):
    """Calculates the blended mAP score from classifier and detector OOF files."""
    print("--- Calculating Blended mAP Score ---")
    
    # --- 1. Calculate Study-level mAP (using per-class AP) ---
    df_oof_cls = pd.read_csv(oof_cls_path)
    df_oof_cls = df_oof_cls.merge(df_merged[['image_id', 'StudyInstanceUID']].drop_duplicates(), on='image_id')
    
    # Aggregate to study level
    df_study_preds = df_oof_cls.groupby('StudyInstanceUID').agg({
        'Negative for Pneumonia': 'min',
        'Typical Appearance': 'max',
        'Indeterminate Appearance': 'mean',
        'Atypical Appearance': 'max'
    }).reset_index()
    
    cols = ['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']
    df_study_preds[cols] = df_study_preds[cols].clip(0,1)
    s = df_study_preds[cols].sum(axis=1).replace(0,1)
    df_study_preds[cols] = df_study_preds[cols].div(s, axis=0)

    # Get ground truth study labels
    df_gt_study = df_merged[['StudyInstanceUID'] + cols].drop_duplicates()
    # FIX: Add suffixes to prevent column name collision during merge
    df_eval_study = df_study_preds.merge(df_gt_study, on='StudyInstanceUID', suffixes=('_pred', '_gt'))
    
    gt_cols = [c + '_gt' for c in cols]
    pred_cols = [c + '_pred' for c in cols]
    
    gt = df_eval_study[gt_cols].values
    pred = df_eval_study[pred_cols].values
    
    study_map = np.mean([average_precision_score(gt[:, i], pred[:, i]) for i in range(4)])
    print(f"Study-level mAP (AP): {study_map:.4f}")
    
    # --- 2. Calculate Image-level mAP ---
    with open(oof_det_path, 'rb') as f: oof_det = pickle.load(f)
    
    gt_boxes = []
    pred_boxes = []
    
    # Get ground truth boxes
    df_gt_image = df_merged[df_merged['boxes'].notna()].copy()
    df_gt_image['boxes'] = df_gt_image['boxes'].apply(eval)
    gt_map = {row.image_id: [[b['x'], b['y'], b['x']+b['width'], b['y']+b['height']] for b in row.boxes] for row in df_gt_image.itertuples()}
    
    for image_id in oof_det.keys():
        # Ground truth: [image_id, class_id, x1, y1, x2, y2]
        if image_id in gt_map:
            for box in gt_map[image_id]:
                gt_boxes.append([image_id, 0, box[0], box[1], box[2], box[3]])
        
        # Predictions: [image_id, class_id, confidence, x1, y1, x2, y2]
        det_preds = oof_det[image_id]
        for box, score in zip(det_preds['boxes'], det_preds['scores']):
            pred_boxes.append([image_id, 0, score, box[0], box[1], box[2], box[3]])
            
    image_map, _ = mean_average_precision_for_boxes(np.array(gt_boxes), np.array(pred_boxes))
    print(f"Image-level mAP: {image_map:.4f}")
    
    # --- 3. Blend Scores ---
    blended_map = (study_map + image_map) / 2.0
    print(f"\nBlended CV Score: {blended_map:.4f}")
    return blended_map

print("Blended mAP calculation function defined.")

🔧 Intercepting pip install command: pip install -q map_boxes


✅ Package installation completed and import cache refreshed.
Blended mAP calculation function defined.


In [25]:
# --- Run Blended mAP Calculation ---
blended_cv_score = calculate_blended_map(
    df_merged=df_merged,
    oof_cls_path='oof_classifier.csv',
    oof_det_path='oof_detector.pkl'
)

--- Calculating Blended mAP Score ---
Study-level mAP (AP): 0.3535


Number of files in annotations: 3879
Number of files in predictions: 5696
Unique classes: 1


Detections length: 5696
Annotations length: 3879


0                              | 0.002562 |    7094
mAP: 0.002562
Image-level mAP: 0.0026

Blended CV Score: 0.1780


In [26]:
# --- Sanity Check: Classifier OOF Diversity ---
import pandas as pd
import numpy as np

print("--- Checking Classifier OOF Diversity ---")
try:
    df_oof_cls = pd.read_csv('oof_classifier.csv')
    
    cols = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
    
    print("\nPrediction Statistics (min/max/std):")
    print(df_oof_cls[cols].agg(['min', 'max', 'std']))
    
    print("\nArgmax Distribution:")
    df_oof_cls['predicted_class'] = df_oof_cls[cols].idxmax(axis=1)
    print(df_oof_cls['predicted_class'].value_counts(normalize=True))
    
except FileNotFoundError:
    print("ERROR: oof_classifier.csv not found.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Checking Classifier OOF Diversity ---

Prediction Statistics (min/max/std):
     Negative for Pneumonia  Typical Appearance  Indeterminate Appearance  \
min                0.000221            0.002174                  0.002174   
max                0.983000            0.997600                  0.751500   
std                0.173763            0.196393                  0.068788   

     Atypical Appearance  
min             0.000124  
max             0.313200  
std             0.039931  

Argmax Distribution:
predicted_class
Typical Appearance          0.780021
Negative for Pneumonia      0.204881
Indeterminate Appearance    0.015098
Name: proportion, dtype: float64


In [ ]:
# --- DEBUG: Read pycocotools source ---
COCO_PY_PATH = '/app/.pip-target/pycocotools/coco.py'
print(f"--- Reading content of {COCO_PY_PATH} ---")
try:
    with open(COCO_PY_PATH, 'r') as f:
        content = f.read()
    print(content)
except FileNotFoundError:
    print(f"ERROR: File not found at {COCO_PY_PATH}")
print("--- End of file content ---")

In [ ]:
# --- Patch pycocotools/coco.py to prevent KeyError ---

COCO_PY_PATH = '/app/.pip-target/pycocotools/coco.py'
print(f"Attempting to patch {COCO_PY_PATH}...")

try:
    with open(COCO_PY_PATH, 'r') as f:
        original_content = f.read()

    # This is the line that causes the error according to the traceback.
    line_to_replace = "res.dataset['info'] = copy.deepcopy(self.dataset['info'])"
    
    # We will find the line and wrap it in a try-except block to handle the error gracefully.
    lines = original_content.split('\n')
    found_and_patched = False
    for i, line in enumerate(lines):
        if line.strip() == line_to_replace:
            indentation = line[:line.find(line.strip())]
            # Wrap the original line in a try-except block, preserving indentation
            patched_block = (
                f"{indentation}try:\n" +
                f"{indentation}    {line.strip()}\n" +
                f"{indentation}except KeyError:\n" +
                f"{indentation}    pass"
            )
            lines[i] = patched_block
            found_and_patched = True
            break
            
    if found_and_patched:
        patched_content = '\n'.join(lines)
        with open(COCO_PY_PATH, 'w') as f:
            f.write(patched_content)
        print(f"SUCCESS: Patched {COCO_PY_PATH} by wrapping the problematic line in a try-except block.")
    else:
        print(f"!!! ERROR: Could not find the line '{line_to_replace}' to patch.")

except Exception as e:
    print(f"An error occurred during patching: {e}")

In [ ]:
# --- Force Clean YOLOX Directories ---
import os

print("Forcefully removing 'yolox_data' and 'yolox_outputs' directories...")
!rm -rf yolox_data
!rm -rf yolox_outputs
print("Cleanup complete. The next cells will regenerate these directories from scratch.")

In [ ]:
# --- Kernel Sanity Check ---
print('Hello from Cell 23!')